In [23]:
import os
from dotenv import load_dotenv
# 获取向量数据库
from langchain_chroma import Chroma
from langchain_ollama import ChatOllama, OllamaEmbeddings, OllamaLLM

load_dotenv()
# 1.加载模型
chat_model = ChatOllama(
    base_url=os.environ['OLLAMA_BASE_URL'],  # 可配置为内部服务器地址
    model="deepseek-r1:32b",
    temperature=0.8,
    num_ctx=4096  # 上下文窗口大小
)
embedding_model = OllamaEmbeddings(
    base_url=os.environ['OLLAMA_BASE_URL'],  # 可配置为内部服务器地址
    model=os.environ['OLLAMA_EMBEDDING_MODEL_NAME'],
    temperature=0.8,
    num_ctx=4096  # 上下文窗口大小
)
llm = OllamaLLM(base_url=os.environ['OLLAMA_BASE_URL'], model="qwq:32b")
vector_store = Chroma(persist_directory="../vector_store/knowledge_base_one", embedding_function=embedding_model)

In [ ]:
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
# 初始化检索器与重排组件
# 基础检索器（如向量库检索器）
# base_retriever = vector_store.as_retriever(
#     search_type="similarity",
#     search_kwargs={"k": 3, "filter": None}
# )
# 混合检索
base_retriever = vector_store.as_retriever(
    search_type="mmr",  # 最大边际相关性
    search_kwargs={"k": 5, "lambda_mult": 0.7}
)
prompt = ChatPromptTemplate.from_messages([
    ("system", "基于以下文本回答问题:\n{context}"), 
    ("human", "{query}")
])
# RunnableParallel构建可执行的并行连，RunnablePassthrough构建可执行的单个链, 链的关键是将上一个执行完过程返回的内容作为下一个链的输入
parallel_chain = RunnableParallel(
        query=RunnablePassthrough(), 
        context=base_retriever | (lambda docs: "\n".join([doc.page_content for doc in docs])))

chain = parallel_chain | prompt | chat_model | StrOutputParser()

results = chain.invoke("全过程民主")
print(results)

<think>
嗯，用户问的是“全过程民主”，根据提供的文本，我需要详细解释这个概念。首先，我应该看看文本中提到的关键词和知识点。

我记得文本里提到了“全链条、全方位、全覆盖”这三个方面，所以我要分别解释每个部分。还有，全过程人民民主是社会主义民主政治的本质属性，这一点也很重要。

接下来，我要确保回答结构清晰，分点说明每个特点，并且用简洁的语言让用户容易理解。另外，可能用户对协商民主也有兴趣，所以在最后提到它作为实现全过程民主的重要形式，会比较全面。

用户可能是在学习相关课程或者写论文，所以详细但不过于学术化的解释会更合适。确保涵盖所有知识点，并引用文本中的例子，比如选举、国家和社会生活的贯通等，这样更有说服力。
</think>

全过程民主是指社会主义民主政治的本质属性和实现形式，强调在国家治理和社会生活中实现人民当家作主的广泛性、真实性和有效性。根据提供的文本内容，全过程民主具有以下特点：

1. **全链条的民主**  
   全过程民主贯穿于民主选举、民主协商、民主决策、民主管理和民主监督等各个环节。例如，在民主选举环节，人民通过选票行使权利，选出代表自己意愿的人来掌握并行使权力。

2. **全方位的民主**  
   全过程民主贯通国家政治生活和社会生活的各层面和维度。它不仅体现在政治领域，还延伸到经济、文化、社会等各个方面，确保人民群众在广泛的社会事务中都能参与决策和管理。

3. **全覆盖的民主**  
   全过程民主涵盖国家各项事业和工作，贯穿“五位一体”总体布局和“四个全面”战略布局的方方面面。它不仅是单一领域的民主，而是各方面民主的有机统一。

4. **最广泛、最真实、最管用的民主**  
   全过程民主是全体人民都能参与和受益的民主形式，确保人民群众在国家政治生活和社会生活中有广泛的参与权、表达权和监督权。

此外，全过程民主也体现在协商民主中。社会主义协商民主是在中国共产党领导下，围绕改革发展稳定的重大问题和涉及群众切身利益的实际问题，在决策之前和决策实施之中开展广泛协商，努力形成共识的重要民主形式。这种民主形式体现了中华民族优秀政治文化传统、中国特色社会主义政治制度的优势以及实践创新成果。

总之，全过程民主是中国特色社会主义政治发展道路的重要体现，是实现人民当家作主的根本保证。


In [22]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_core.documents.compressor import BaseDocumentCompressor
from typing import List
from langchain_core.documents import Document
from typing import TYPE_CHECKING, Optional

class OllamaReranker(BaseDocumentCompressor):
    def __init__(self):
        print("OllamaReranker init")

    def compress_documents(self, documents: List[Document], query: str, callbacks=None) -> List[Document]:
        # 生成相关性评分并排序
        scored_docs = []
        for doc in documents:
            prompt = f"""评估文本与问题的相关性（0-10）：
            问题：{query}
            文本：{doc.page_content}
            评分："""
            score = llm.invoke(prompt).strip()
            scored_docs.append((score, doc))
            print(f"{doc.page_content} 评分：{score}")
        
        # 按评分降序排列
        sorted_docs = sorted(scored_docs, key=lambda x: x[0], reverse=True)
        return [doc for _, doc in sorted_docs]

compressor = OllamaReranker()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=base_retriever
)

# # 使用重排后的检索器
reranked_docs = compression_retriever.invoke("全过程民主")
result = "\n".join([doc.page_content for doc in reranked_docs])
print(result)

OllamaReranker init
[history] 第八章 发展全过程人民民主 评分：<think>

好的，我现在要评估用户提供的文本和问题之间的相关性，并给出一个0到10分的评分。首先，我需要明确问题是什么以及文本的内容。

问题：“全过程民主”是用户关注的核心关键词。接下来要看文本内容是否与此主题直接相关。根据用户提供的信息，文本标题是“第八章 发展全过程人民民主”。这里的关键词包括“发展”、“全过程”和“人民民主”。

首先，“全过程人民民主”明显包含了问题中的“全过程民主”，只是多了一个“人民”二字。这说明文本直接讨论的是与问题相关的主题，但可能更具体地涉及了中国的语境或实践。因此，从字面上看，两者的相关性很高。

接下来需要分析是否还有其他因素会影响评分。例如，如果文本详细阐述了全过程民主的定义、实施方法、历史背景或者案例，那么相关性会更高；反之，如果只是提到这个概念而没有深入讨论，则可能稍低一些。不过用户提供的文本只是章节标题，并未给出具体内容，所以只能基于标题进行判断。

考虑到标题明确提到了“发展”和“全过程人民民主”，可以推断这一章很可能详细探讨了该主题的各个方面，如理论基础、实践措施等，因此相关性应该很高。此外，“全过程民主”是当前中国政治领域的重要概念，与文本中的章节标题完全匹配，所以关联度很强。

可能存在的扣分点：虽然“人民”一词添加了限定，但问题并未排除特定国家或地区的语境，因此这并不影响评分。相反，它进一步明确了主题的具体应用范围，反而增加了相关性。

综合来看，这个文本直接对应了用户的问题，没有偏离主题，并且是专门讨论该主题的章节，所以应该给予较高的分数。通常在这种情况下，如果完全匹配，可能给9或10分。考虑到可能存在细微差异（如“人民”是否影响问题的泛化程度），但问题本身并未限制范围，因此可以认为是直接相关。

最终评分应该是10分，因为标题与问题高度一致，并且明确涉及全过程民主的发展，没有其他不相关的因素干扰。
</think>

根据文本标题和问题的对比分析，给出以下评估：

**评分：10/10**

**理由：**
1. **直接关键词匹配**  
   文本标题“发展全过程人民民主”与问题“全过程民主”核心词汇完全一致（“全过程民主”是问题的核心），仅增加了限定词“人民”，但未偏离主题。

2. **明确的主题聚